In [6]:
import pandas as pd

# Charger les fichiers
df_rest = pd.read_csv("restaurants_paris_clean.csv")
df_indice = pd.read_csv("comm-idf-dataiss.csv")

# Nettoyage des colonnes pour jointure
df_rest['code_postal'] = df_rest['code_postal'].astype(str).str.extract(r'(\d{5})')  # garde les 5 chiffres si présents
df_rest['code_postal'] = df_rest['code_postal'].fillna("00000")  # sécurité

df_indice['INSEE_COM'] = df_indice['INSEE_COM'].astype(str).str.zfill(5)


# Conversion code postal -> code INSEE (cas Paris)
def convert_cp_to_insee(cp):
    if cp.startswith("75"):
        # 75018 -> 75118
        return "751" + cp[-2:]
    else:
        # pour les communes hors Paris, CP = INSEE (si c'est le cas dans ton fichier)
        return cp

df_rest["INSEE_FROM_CP"] = df_rest["code_postal"].apply(convert_cp_to_insee)

# Jointure
df_merged = df_rest.merge(
    df_indice,
    left_on="INSEE_FROM_CP",
    right_on="INSEE_COM",
    how="left"
)


# Sauvegarde du résultat
df_merged.to_csv("restaurants_agg.csv", index=False)
